## patchづくり&csvづくり

きれいなディレクトリ構造で、明瞭に条件分岐できるようにデータを作成する  
パッチとcsvをセットで保存しておいて、一般性や再現性を保ったパッチを作っていきたい  

手順としては  
1. patch用の関数を通す   
2. csvが作られる  
3. 条件分岐が行われる  

条件
- 対象のnii.gzの名前が変わっても大丈夫
- 患者単位のstatics.csvがあってもいいかも

In [1]:
import math
import SimpleITK as sitk
import numpy as np
import pathlib
from tqdm import tqdm
import numpy.ma as ma
import glob
from pathlib import Path


In [4]:
impath = '/home/higuchi/ssd/kits19/data/case_00000/imaging.nii.gz'
mapath = '/home/higuchi/ssd/kits19/data/case_00000/segmentation.nii.gz'
im = sitk.ReadImage(impath)
ma = sitk.ReadImage(mapath)
im_arr = sitk.GetArrayFromImage(im)
ma_arr = sitk.GetArrayFromImage(ma)



In [7]:
read_range = im.GetSize()
read_range

(101, 602, 602)

In [10]:
def getListCropPoint(read_range, pad_range):
    # read_range：パッチの始点の範囲
    # pad_range：始点の基準間隔
    equal = math.ceil(read_range / pad_range)
    crop_point = np.round(np.linspace(0, read_range, equal + 1)).astype(int)

    return crop_point

size = [16,48,48]
overlap = [2,3,3]
crop_points = [[getListCropPoint(read_range[i] - size[i],size[i]//overlap[i])] for i in range(3)]


In [ ]:
def make_patch(x, y, z,size,save_dir):
    #　ラベルはネットワークの出力パッチサイズの大きさでクロップする
    crop_label = label_array[x:x + size[0], y:y + size[1], z:z + size[2]]
    
    # パッチ作成の条件
    if (crop_label != 0).sum() <= np.prod(size) * 0.001 or\
            (crop_label != 0).sum() >= np.prod(size) * 0.8:
        return

    # TODO:時相が複数のときに対応させる
    crop_vol = image_array[x:x + size[0], y:y + size[1], z:z + size[2]]

    patch_name = f'{str(x).zfill(3)}_{str(y).zfill(3)}_{str(z).zfill(3)}.npy'
    save_vol_path = save_directory /'image'/ patch_name
    save_seg_path = save_directory /'lable'/ patch_name

    saveNPY(crop_label, str(save_seg_path))
    saveNPY(crop_vol, str(save_vol_path), True)


In [ ]:
for z in tqdm(crop_points[0]):
    for y in crop_points[1]:
        for x in crop_points[2]:
            patch_methos(z,y,x)

In [ ]:
patch_name = 'normal'
file_name_dict = {
    'image':['imaging.nii.gz'],
    'mask':['segmentation.nii.gz']
}

def make_patch(niigz_path,save_dir,size=[48,48,16],)